<a href="https://colab.research.google.com/github/ethancpwoo/Practice-NeuralNetwork/blob/main/DogsvCats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
